# Answering the Question: What characters does The Data say I should use?
## Author: Kevin Stull


In [1]:
# libraries
import pandas as pd
import matplotlib.pyplot as plt

# load data
df_char_bases = pd.read_csv('fe_17_data_char_bases.csv')
df_char_growths = pd.read_csv('fe_17_data_char_growths.csv')
temp = pd.merge(df_char_bases, df_char_growths[['Name', 'Ch_Rec']], on='Name', how='left')
df_char_bases['Ch_Rec'] = temp['Ch_Rec'] 
df_char_bases.describe()
df_char_bases

,Name,HP,Str,Mag,Dex,Spd,Def,Res,Lck,Bld,Internal Level,Total,Total Without HP,Ch_Rec
0,Veyle,1,8,4,8,8,6,10,8,3,0,56,55,22
1,Saphir,4,3,1,10,4,7,5,8,4,19,46,42,19
2,Mauvier,7,3,6,4,3,6,6,6,1,20,42,35,21
3,Goldmary,3,3,1,5,6,3,5,5,1,17,32,29,16
4,Lindon,3,1,4,7,3,3,3,2,3,18,29,26,18
5,Merrin,3,4,2,4,1,2,2,5,1,15,24,21,13
6,Kagetsu,3,5,1,3,1,2,2,5,1,15,23,20,11
7,Ivy,3,2,4,7,3,1,1,1,0,0,22,19,11
8,Panette,3,3,1,5,2,2,2,5,-1,15,22,19,13
9,Zelkov,3,4,1,3,3,2,1,1,3,0,21,18,11


## Introduction
Fire Emblem Engage presents us with a simple problem, given 36 possible characters, which 10 or so should we put on our team? Should I just commit to the weaker early game character or hold off for this character in the late-game? What characters are best? In order to answer this question we are going to make a few assumptions. Number 1, we will assume that the classes are balanced, this means that a lvl 10 swordmaster is just as strong as a lvl 10 mage. While this assumption is almost certainly flawed, it gives us a nice starting point for our investigation which we can remove in later articles. 2nd, we are going to assume that every unit will have average growths. We could also investigate when certain characters become better or worse given random level ups, or just the level up schedule Nintendo provided, but for now, just to simplify things, we assume that the units always perfectly follow their growth curves. Lastly, we will ignore their innate skills. Obviously some characters have better innate abilities that others, but it isn't clear how one would quantify them numerically, so we will assume they are all even and in the case of ties, their passive can be used as a "tie-breaker". With that, we are ready to dive in! 



## What We Are Going To Do 
There are three things that we need to consider; bases, growths, and availability. The first two have been investigated in article 1. The tricky part is deciding how to deal with the character's availability. For example, in the first article we saw that Mavier had the highest total growths and the highest total bases, however he was the worst character in terms of availability, being the last obtainable character on the roster of 36. Simply stating that Mavier is the best tank and moving on does not help us, because we will have to complete most of the game before even getting to him, so what characters should we have used in the mean time. On the other end of the spectrum, barring Alear, most of the characters obtained very early are quite poor (Etie, Bucheron, ect.). The first characters that seemed to be worth considering were Alfred and Citrine, which isn't suprising since they are "plot-relevant" characters with unique promotions. However, at the same time, we meet characters like Louis and Chloe, some more fan favorites. To address this we are going to leverage a machine learning technique called regression. 

## What is Regression and Why Will it Help?
Regression is a pretty simple concept with some pretty powerful applications. Let us start by imagining that we have a graph of all the characters in the game where the x-axis represents what chapter you can recruit them and y represents how powerful they are. We saw from article 1 that later on in the game, characters tend to have more bases and better growths, thus as x increases so does y.

To perform regression, we will try to find constants A, B, and C such that $Ax^2 + Bx + C$ predicts about how strong (y) a unit will be at some chapter (x) in the game. 

I selected a quadratic (function with x to the power of 2) because that is what the curves roughly looked like in article 1. Once we have this line that predicts how strong we expect a unit to be at some chapter, we will look at which characters are the furthest above that curve. That would mean that these characters are much stronger than we would expect a character to be at that chapter of the game. Thus, giving XP to them will on average lead to a stronger character later on in the game even if we encounter another unit that might fill the same role. 

For example, in article 1 we saw that Mauvier, Goldmary, and Louis had the best growths for tanking. Let's say that our analysis finds that Goldmary is the best unit of the three. In order to get through the early game, we might have to use Luis for a while and he might get some XP, we should still switch to Goldmary when we unlock her because she has a better combination of starting stats and growths, so when they reach the same level (promotion level 1), we still expect that Goldmary will end up being stronger. Since Goldmary starts at level 1, we ought to just upgrade to her. Further, when Mauvier is unlocked, we should not switch because we expect that Goldmary will end up being a stronger unit when she reaches the same level as Mauvier. 

It is worth noting that this assumes that Goldmary will be the same level as (or a higher level than) Mauvier when we meet him. If she were a lower level then it might still be worth switching over because of his superior growths and higher bases. I am ignoring this option because an optimal strategy implies wasting as little XP as possible on characters that will not be in the final team (you want to give Vander as little XP as possible), so in general, your units should be a higher level than the units you can recruit on any given chapter. NOTE: this assumption has not been tested on Maddending or above, need input from an experienced player. 

## Basic Approach 
1) Create a graph containing all the characters where x is the chapter you can recruit them and y is their power level. 
2) Use regression to find a best-fit quadrati function to the data in that graph 
3) Rank all of the character's desirability in terms of how far above the expected powere level line they are 
4) Select the top N characters to build your team (where N is the size of your team)